# PD 6
## Piotr Fic
### Import bibliotek

In [1]:
import numpy as np
from numpy import random
import math
import pandas as pd
import matplotlib.pyplot as plt

## Funkcje pomocnicze

### Aktywacja

In [2]:
def softmax(y):
    exp = np.exp(y-np.max(y))
    return exp/exp.sum(axis = 0, keepdims=True)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_der(x):
    return sigmoid(x)*(1-sigmoid(x))

def linear(x):
    return x

def linear_der(x):
    return 1

def tanh(x):
    exp = np.exp(x)
    exp_ = np.exp(-x)
    return (exp - exp_)/(exp + exp_)

def tanh_der(x):
    return (1-tanh(x)**2)

def ReLu(x):
    return np.maximum(0, x)

def ReLu_der(x):
    return np.where(x>0, 1, 0)

### Miary

In [3]:
def MSE(predicted, real):
    return np.mean((predicted - real)**2)

def MAE(predicted, real):
    return np.mean(np.abs(predicted - real))

def acc(x, y):
    return (x==y).mean()

### Wykresy i dane

In [218]:
def one_hot(y):
    y = y.astype(int)
    n_values = np.max(y) + 1
    return np.eye(n_values)[y.T][0]

def prepear_classif(df_train_p, df_test_p):
    df_train = pd.read_csv(df_train_p)
    df_test = pd.read_csv(df_test_p)
    #Separacja wektorów cech i odpowiedzi
    x_train, x_test = df_train.iloc[:, 0:2], df_test.iloc[:, 0:2]
    x_train, x_test = np.array(x_train), np.array(x_test)
    y_train, y_test = np.array(df_train.iloc[:, 2:3]), np.array(df_test.iloc[:, 2:3])
    #Normalizacja
    x_train, x_test = normalize(x_train), normalize(x_test)
    return x_train, one_hot(y_train), x_test, y_test

def prepear_data(df_train_p, df_test_p):
    df_train = pd.read_csv(df_train_p)
    df_test = pd.read_csv(df_test_p)
    #Separacja wektorów cech i odpowiedzi
    x_train, x_test = df_train.iloc[:, 1], df_test.iloc[:, 1]
    x_train, x_test = np.array(x_train), np.array(x_test)
    y_train, y_test = np.array(df_train.iloc[:, 2:3]), np.array(df_test.iloc[:, 2:3])
    #Normalizacja
    x_train, x_test = normalize(x_train), normalize(x_test)
    y_train, y_test = normalize(y_train), normalize(y_test)
    return x_train.reshape(-1, 1), y_train.reshape(-1, 1), x_test.reshape(-1, 1), y_test.reshape(-1, 1)

In [5]:
def predicted_real(predicted, real):
    plt.scatter(real, predicted, c='blue')
    p1 = max(max(predicted), max(real))
    p2 = min(min(predicted), min(real))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('True values')
    plt.ylabel('Predictions')
    plt.title('Predictions visualization on test set')
    plt.axis('equal')
    plt.show()

def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

## Implementacja MLP
 - dodanie uczenia z karą L2
 - implementacja uczenia ze zbiorem walidacyjnym

In [6]:
class Layer:
        
    def __init__(self, 
                 #Liczba neuronów w poprzedzającej i kolejnej warstwie
                 input_size: int, 
                 output_size: int,
                 #Domyślna funkcja aktywacyjna i jej pochodna                 
                 activation_fun = sigmoid, 
                 activation_fun_der = sigmoid_der,
                 ):
        
        #Wagi dla wszystkich neuronów - inicjalizacja Xavier
        self.biases = np.random.uniform(
            -math.sqrt(6)/ math.sqrt(input_size + output_size),
            math.sqrt(6)/ math.sqrt(input_size + output_size),
                    size=(output_size, 1)
                )
        
        self.weights = np.random.uniform(
            -math.sqrt(6)/ math.sqrt(input_size + output_size),
            math.sqrt(6)/ math.sqrt(input_size + output_size),
                    size=(output_size, input_size)
                )
            
        #Funkcja aktywacji i jej pochodna
        self.activation_fun = activation_fun
        self.activation_fun_der = activation_fun_der
    
    def predict(self, input):
        #Przekształca input z poprzedniej warstwy przez wagi i funkcję aktywacji 
        #Zwraca output do przekazania kolejnej warstwie
        return self.activation_fun(self.weights@input + self.biases)
    
    def forward(self, input):
        #Przekształca input z poprzedniej warstwy jedynie przez wagi
        return self.weights@input + self.biases

In [44]:
class Network:
    
    def __init__(self, layers: list, classif: bool):
        #classif
        self.classif = classif
        #Warstwy
        self.layers = layers
        self.momentum_w = [np.zeros(l.weights.shape) for l in layers]
        self.momentum_b = [np.zeros(l.biases.shape) for l in layers]
        self.g_w = [np.zeros(l.weights.shape) for l in layers]
        self.g_b = [np.zeros(l.biases.shape) for l in layers]
        
        if classif:
            self.layers[-1].activation_fun = softmax
        
    def predict(self, X):
        """Oblicza output na podstawie danych i parametrów warstw"""
        output = X.T
        for layer in self.layers:
            output = layer.predict(output)
        return output.T
    
    def forward(self, X):
        """Pełna metoda feedforward
        return: sumy, aktywacje"""
        sums = []
        activations = [X]
        activations_layer = X
        for layer in self.layers:
            sums_layer = layer.forward(activations_layer)
            sums.append(sums_layer)
            
            activations_layer = layer.activation_fun(sums_layer)
            activations.append(activations_layer)
            
        return sums, activations
    
    def L2_cost(self):
        """Funkcja pomocnicza, licząca części funkcji kosztu"""
        suma = 0
        for i in range(len(self.layers)):
            suma += np.sum(np.square(self.layers[i].weights))
        return suma
    
    def backprop(self, X, Y, l2):
        """Propagacja wsteczna błędu
        return: gradienty MSE"""
        #Batch size
        batch_size = Y.shape[1]
        
        #Wyliczenie feedforward obecnymi parametrami
        sums, activations = self.forward(X)
        
        #Macierze na poprawki parametrów
        delta_biases = []
        delta_weights = []
        
        n_layers = len(self.layers)
        err = [None]*n_layers
        
        #Wyliczenie err ostatniej warstwy
        if self.classif:
            err[-1] = Y - activations[-1]
        else:
            err[-1] = (Y - activations[-1])*self.layers[-1].activation_fun_der(sums[-1])
        
        #Regularyzacja L2
        err[-1] = err[-1] + l2*self.L2_cost()/(2*batch_size)
        
        #Propagacja wsteczna
        for i in reversed(range(len(err) -1)):
            act_f_der = self.layers[i].activation_fun_der(sums[i])
            err[i] = (self.layers[i+1].weights.transpose()@err[i+1]) * act_f_der 
                        
        delta_biases = [e@np.ones((batch_size, 1))/float(batch_size) 
                        for e in err]
        delta_weights = [np.dot(e, activations[i].transpose())/float(batch_size) + l2*self.layers[i].weights/batch_size
                        for i, e in enumerate(err)]
        
        return delta_biases, delta_weights
    
    def train(self, X, Y, 
              batch_size = 1, 
              etha = 0.001, 
              tol = 10**(-6), 
              n_iter = 500,
              #Regularyzacja L2
              l2 = 0,
              #Lambda dla momentum
              l_m = 0,
              #Beta dla RMSProp
              beta = 1,
              verbose = False):
        
        it = 0
        while it<n_iter:
            #Losowość przed podziałem zbioru, ziarno zapewnia identyczną permutację
            #w zmiennych objaśnających i zmiennej celu
            idx = random.permutation(X.shape[0])
            X_train = np.copy(X[idx])
            Y_train = np.copy(Y[idx])
            
            self.batch_gd(X_train, Y_train, batch_size, etha, l_m, beta, l2)
            
            #Wizualizacja procesu uczenia
            if(verbose):
                print("Epoche " + str(it) + " finished")
                for i in range(len(self.layers)):
                    print("Warstwa " + str(i) + " wagi:")
                    print(self.layers[i].weights)
                    print("Warstwa " + str(i) + " bias-y:")
                    print(self.layers[i].biases)
                print("\n")
            
            it += 1
            
        return it
    
    def train_valid(self, X, Y, 
              batch_size = 1, 
              etha = 0.001,
              #Uczenie ze zbiorem walidacyjnym
              score_fun = MSE,      
              tol = 10**(-6), 
              n_iter = 500,
              #Regularyzacja L2
              l2 = 0,
              #Lambda dla momentum
              l_m = 0,
              #Beta dla RMSProp
              beta = 1,
              verbose = False):
        """Trening sieci z kontrolą błędu na zbiorze walidacyjnym"""
        idx = random.permutation(X.shape[0])
        X = np.copy(X[idx])
        Y = np.copy(Y[idx])
            
        sp = int(0.8*len(X))
        X_val = X[sp:]
        Y_val = Y[sp:]
        X = X[:sp]
        Y = Y[:sp]
        
        it = 0
        old_score = 100
        new_score = 10
        while (old_score-new_score)>tol:
            old_score = new_score
            
            #Losowość przed podziałem zbioru, ziarno zapewnia identyczną permutację
            #w zmiennych objaśnających i zmiennej celu
            idx = random.permutation(X.shape[0])
            X_train = np.copy(X[idx])
            Y_train = np.copy(Y[idx])
            
            self.batch_gd(X_train, Y_train, batch_size, etha, l_m, beta, l2)
            
            pred = self.predict(X_val)
            new_score = score_fun(pred, Y_val)
            
            #Wizualizacja procesu uczenia
            if(verbose):
                print("Epoche " + str(it) + " finished")
                for i in range(len(self.layers)):
                    print("Warstwa " + str(i) + " wagi:")
                    print(self.layers[i].weights)
                    print("Warstwa " + str(i) + " bias-y:")
                    print(self.layers[i].biases)
                print("\n")
            
            it += 1
        
        return it
    
    def gd(self, X, Y, etha, l_m, beta, l2):
        """Trening sieci podstawową metodą Gradient Descent"""
        b, w = self.backprop(X, Y, l2)
        for i in range(len(self.layers)):
            #Aktualizacja momentum
            self.momentum_w[i] = w[i]+l_m*self.momentum_w[i]
            self.momentum_b[i] = b[i]+l_m*self.momentum_b[i]
            
            #Aktualizacja RMSProp
            self.g_w[i] = (1-beta)*w[i]*w[i]+beta*self.g_w[i]
            self.g_b[i] = (1-beta)*b[i]*b[i]+beta*self.g_b[i]
        
            l = self.layers[i]
            if(beta):
                l.biases = l.biases + etha*self.momentum_b[i] + (etha*b[i])/np.sqrt(0.00001+self.g_b[i]) 
                l.weights = l.weights + etha*self.momentum_w[i] + (etha*w[i])/np.sqrt(0.00001+self.g_w[i])
            else:
                l.biases = l.biases + etha*self.momentum_b[i]
                l.weights = l.weights + etha*self.momentum_w[i]
            
        return
    
    def batch_gd(self, X, Y, batch_size, etha, l_m, beta, l2):
        """Trening sieci metodą Mini-batch Gradient Descent"""
        #Metoda train uprzednio dokonuje permutacji zbioru
        #Wywołanie metody gradient descent na kolejnych batch-ach
        i=0
        while(i<len(Y)):
            x = X[i:i+batch_size]
            y = Y[i:i+batch_size]
            i += batch_size
            self.gd(x.T, y.T, etha, l_m, beta, l2)
        return
        

# Test działania

### architektura

In [11]:
#1-5-5-1
def arch_2(f, f_d, classif, inp=1, outp=1):
    l1 = Layer(inp, 5, activation_fun=f, activation_fun_der=f_d)
    l2 = Layer(5, 5, activation_fun=f, activation_fun_der=f_d)
    l3 = Layer(5, outp, activation_fun=linear, activation_fun_der=linear_der)
    return Network([l1, l2, l3], classif)

## Regresja
### df: multimodal-sparse

In [262]:
x_train, y_train, x_test, y_test = prepear_data("./mio1/regression/multimodal-sparse-training.csv",
                                               "./mio1/regression/multimodal-sparse-test.csv")

In [275]:
ep = 10
val = [0]*ep
raw = [0]*ep
reg = [0]*ep

for i in range(ep):
    n_val = arch_2(tanh, tanh_der, False)
    n_raw = arch_2(tanh, tanh_der, False)
    n_reg = arch_2(tanh, tanh_der, False)

    #with valid
    n_val.train_valid(x_train, y_train, batch_size=40, etha = 0.0001, tol=10**(-3))
    res = n_val.predict(x_test)
    val[i] = MSE(y_test, res)
    #no valid
    n_raw.train(x_train, y_train, batch_size=40, etha = 0.0001, n_iter=10)
    res = n_raw.predict(x_test)
    raw[i] = MSE(y_test, res)
    #L2
    n_reg.train(x_train, y_train, batch_size=40, etha = 0.0001, n_iter=10, l2 = 0.8)
    res = n_reg.predict(x_test)
    reg[i] = MSE(y_test, res)

In [276]:
result = pd.DataFrame.from_records(np.array([raw, val, reg]).T, columns=["no_reg", "valid_df", "L2_pen"])

#### średnie MSE na zbiorze testowym

In [277]:
result.describe()

,no_reg,valid_df,L2_pen
count,10.000000,10.000000,10.000000
mean,0.156746,0.090242,0.129759
std,0.127329,0.051691,0.059867
min,0.028333,0.017755,0.064654
25%,0.064989,0.061945,0.082422
50%,0.135586,0.078096,0.112867
75%,0.199672,0.103328,0.174530
max,0.459853,0.190587,0.238202


## Klasyfikacja
### df: rings5-sparse

In [330]:
x_train, y_train, x_test, y_test = prepear_classif("./mio1/classification/rings5-sparse-training.csv",
                                               "./mio1/classification/rings5-sparse-test.csv")

In [338]:
ep = 20
val = [0]*ep
raw = [0]*ep
reg = [0]*ep

for i in range(ep):
    n_val = arch_2(tanh, tanh_der, inp=2, outp=5, classif=True)
    n_raw = arch_2(tanh, tanh_der, inp=2, outp=5, classif=True)
    n_reg = arch_2(tanh, tanh_der, inp=2, outp=5, classif=True)

    #with valid
    n_val.train_valid(x_train, y_train, batch_size=200, etha = 0.001, tol=10**(-5))
    res = n_val.predict(x_test)
    pred = np.argmax(res, axis=1)
    val[i] = acc(y_test, pred)
    #no valid
    n_raw.train(x_train, y_train, batch_size=200, etha = 0.001, n_iter=20)
    res = n_raw.predict(x_test)
    pred = np.argmax(res, axis=1)
    raw[i] = acc(y_test, pred)
    #L2
    n_reg.train(x_train, y_train, batch_size=200, etha = 0.001, n_iter=20, l2 = 0.3)
    res = n_reg.predict(x_test)
    pred = np.argmax(res, axis=1)
    reg[i] = acc(y_test, pred)

In [339]:
result = pd.DataFrame.from_records(np.array([raw, val, reg]).T, columns=["no_reg", "valid_df", "L2_pen"])

#### średnie ACC na zbiorze testowym

In [340]:
result.describe()

,no_reg,valid_df,L2_pen
count,20.000000,20.000000,20.000000
mean,0.178839,0.198047,0.191973
std,0.040811,0.048099,0.039097
min,0.082607,0.077964,0.124287
25%,0.149131,0.178914,0.168778
50%,0.192933,0.204859,0.184910
75%,0.200474,0.237113,0.206075
max,0.260362,0.261337,0.289853


### df: rings3-balance

In [341]:
x_train, y_train, x_test, y_test = prepear_classif("./mio1/classification/rings3-balance-training.csv",
                                               "./mio1/classification/rings3-balance-test.csv")

In [386]:
df = pd.read_csv("./mio1/classification/rings3-balance-test.csv")

In [392]:
ep = 5
val = [0]*ep
raw = [0]*ep
reg = [0]*ep

for i in range(ep):
    n_val = arch_2(tanh, tanh_der, inp=2, outp=3, classif=True)
    n_raw = arch_2(tanh, tanh_der, inp=2, outp=3, classif=True)
    n_reg = arch_2(tanh, tanh_der, inp=2, outp=3, classif=True)

    #with valid
    n_val.train_valid(x_train, y_train, batch_size=2060, etha = 0.001, tol=10**(-7))
    res = n_val.predict(x_test)
    pred = np.argmax(res, axis=1)
    val[i] = acc(y_test, pred)
    #no valid
    n_raw.train(x_train, y_train, batch_size=2060, etha = 0.001, n_iter=500)
    res = n_raw.predict(x_test)
    pred = np.argmax(res, axis=1)
    raw[i] = acc(y_test, pred)
    #L2
    n_reg.train(x_train, y_train, batch_size=2060, etha = 0.001, n_iter=500, l2 = 0.6)
    res = n_reg.predict(x_test)
    pred = np.argmax(res, axis=1)
    reg[i] = acc(y_test, pred)

In [393]:
result = pd.DataFrame.from_records(np.array([raw, val, reg]).T, columns=["no_reg", "valid_df", "L2_pen"])

#### średnie ACC na zbiorze testowym

In [394]:
result.describe()

,no_reg,valid_df,L2_pen
count,5.000000,5.000000,5.000000
mean,0.317618,0.310926,0.321991
std,0.001549,0.003446,0.007409
min,0.315827,0.307533,0.312717
25%,0.316675,0.308098,0.319031
50%,0.317335,0.310643,0.320634
75%,0.318466,0.312339,0.324875
max,0.319785,0.316015,0.332697


### df: xor3-balance

In [452]:
x_train, y_train, x_test, y_test = prepear_classif("./mio1/classification/xor3-balance-training.csv",
                                               "./mio1/classification/xor3-balance-test.csv")

In [478]:
ep = 5
val = [0]*ep
raw = [0]*ep
reg = [0]*ep

for i in range(ep):
    n_val = arch_2(tanh, tanh_der, inp=2, outp=2, classif=True)
    n_raw = arch_2(tanh, tanh_der, inp=2, outp=2, classif=True)
    n_reg = arch_2(tanh, tanh_der, inp=2, outp=2, classif=True)

    #with valid
    n_val.train_valid(x_train, y_train, batch_size=40, beta=0.9, l_m=0.9, etha = 0.001, tol=10**(-5))
    res = n_val.predict(x_test)
    pred = np.argmax(res, axis=1)
    val[i] = acc(y_test, pred)
    #no valid
    n_raw.train(x_train, y_train, batch_size=50, beta=0.9, l_m=0.9, etha = 0.001, n_iter=4000)
    res = n_raw.predict(x_test)
    pred = np.argmax(res, axis=1)
    raw[i] = acc(y_test, pred)
    #L2
    n_reg.train(x_train, y_train, batch_size=50, beta=0.9, l_m=0.9, etha = 0.001, n_iter=4000, l2 = 0.0001)
    res = n_reg.predict(x_test)
    pred = np.argmax(res, axis=1)
    reg[i] = acc(y_test, pred)

In [479]:
result = pd.DataFrame.from_records(np.array([raw, val, reg]).T, columns=["no_reg", "valid_df", "L2_pen"])

#### średnie ACC na zbiorze testowym

In [480]:
result.describe()

,no_reg,valid_df,L2_pen
count,5.000000,5.0000,5.000000
mean,0.529846,0.5525,0.535884
std,0.003703,0.0000,0.011980
min,0.524675,0.5525,0.520606
25%,0.527956,0.5525,0.528613
50%,0.529925,0.5525,0.534650
75%,0.533075,0.5525,0.546462
max,0.533600,0.5525,0.549087


## Wnioski
Dla części zbiorów możemy zaobserwować poprawę wyników po regularyzacji na zbiorze testowym. Z drugiej strony osiągane miary są ogółem dość słabe w związku z utrudnioną strukturą zbiorów i małymi rozmiarami części treningowych względem testowych. 